In [20]:
#Import the basics
import numpy as np
import matplotlib as plt
import pandas as pd

#Get sklearn
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut

#Get tensorflow and keras
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import optimizers
# Helper libraries

print(tf.__version__)

2.6.0


In [21]:
#Read in data from the csv file
data = pd.read_csv("Structured_Data.csv")

In [22]:
#Check things look good
data.head(3)

,Target-20_Open_VET,Target-20_High_VET,Target-20_Low_VET,Target-20_Close_VET,Target-20_Volume_VET,Target-20_Open_BTC,Target-20_High_BTC,Target-20_Low_BTC,Target-20_Close_BTC,Target-20_Volume_BTC,...,Target-1_Open_BTC,Target-1_High_BTC,Target-1_Low_BTC,Target-1_Close_BTC,Target-1_Volume_BTC,Target-1_SMA20,Target-1_Upper_Bollinger_Band,Target-1_Lower_Bollinger_Band,Regression_Target,Classification_Target
0,0.014743,0.014743,0.013981,0.014535,19980000,6551.520020,6719.959961,6498.640137,6719.959961,4097820000,...,6317.009766,6363.870117,6265.089844,6351.799805,4064230000,0.016273,0.019782,0.012764,0.014060,1.0
1,0.014552,0.015735,0.014244,0.015413,24453700,6719.950195,6789.629883,6700.959961,6763.189941,3312600000,...,6354.240234,6535.410156,6354.240234,6517.310059,4210910000,0.016249,0.019811,0.012688,0.013978,0.0
2,0.015390,0.015446,0.014513,0.015233,20503900,6754.640137,6774.750000,6620.750000,6707.259766,3295500000,...,6515.410156,6596.100098,6456.169922,6512.709961,4076220000,0.016178,0.019857,0.012499,0.014275,1.0


In [23]:
#Create target column for regression
y = data["Classification_Target"]
y

0       1.0
1       0.0
2       1.0
3       0.0
4       0.0
       ... 
1139    1.0
1140    0.0
1141    0.0
1142    1.0
1143    0.0
Name: Classification_Target, Length: 1144, dtype: float64

In [24]:
#Drop the target columns to create input data
X = data.drop(columns=["Regression_Target", "Classification_Target"])
X.head(3)

,Target-20_Open_VET,Target-20_High_VET,Target-20_Low_VET,Target-20_Close_VET,Target-20_Volume_VET,Target-20_Open_BTC,Target-20_High_BTC,Target-20_Low_BTC,Target-20_Close_BTC,Target-20_Volume_BTC,...,Target-1_Close_VET,Target-1_Volume_VET,Target-1_Open_BTC,Target-1_High_BTC,Target-1_Low_BTC,Target-1_Close_BTC,Target-1_Volume_BTC,Target-1_SMA20,Target-1_Upper_Bollinger_Band,Target-1_Lower_Bollinger_Band
0,0.014743,0.014743,0.013981,0.014535,19980000,6551.520020,6719.959961,6498.640137,6719.959961,4097820000,...,0.013835,25123900,6317.009766,6363.870117,6265.089844,6351.799805,4064230000,0.016273,0.019782,0.012764
1,0.014552,0.015735,0.014244,0.015413,24453700,6719.950195,6789.629883,6700.959961,6763.189941,3312600000,...,0.014060,28222000,6354.240234,6535.410156,6354.240234,6517.310059,4210910000,0.016249,0.019811,0.012688
2,0.015390,0.015446,0.014513,0.015233,20503900,6754.640137,6774.750000,6620.750000,6707.259766,3295500000,...,0.013978,16430800,6515.410156,6596.100098,6456.169922,6512.709961,4076220000,0.016178,0.019857,0.012499


In [25]:
#Split the data into training (80%) and testing sets (20%), includes shuffling
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2)

In [26]:
#Split the training data into training (80%) and validation (20%)
xtraintrain, xval, ytraintrain, yval = train_test_split(xtrain, ytrain, test_size=0.2)

In [40]:
def mapto1hot(ydata):

    ydata_1hot = tf.keras.utils.to_categorical(ydata, num_classes=2)

    return ydata_1hot

In [41]:
ytrain_1hot = mapto1hot(ytrain)
ytest_1hot = mapto1hot(ytest)

In [44]:
layers = [
    Flatten(),
    Dense(10000, activation='relu', kernel_regularizer='l1'),
    Dense(1000, activation='relu', kernel_regularizer='l1'),
    Dense(1000, activation='relu', kernel_regularizer='l1'),
    Dense(1000, activation='relu'),
    Dense(1000, activation='relu'),
    Dense(2, activation='softmax')
]

model = Sequential(layers)

opt = tf.keras.optimizers.SGD(learning_rate=0.01)
#refer to https://www.tensorflow.org/tutorials/keras/classification tutorial to check on how to use compile function
model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=['accuracy'])

# This will start the training and save each epoch output in the history list
history = model.fit(xtrain, ytrain_1hot, batch_size=10, epochs=20, validation_data=(xtest, ytest_1hot))

Epoch 1/20
92/92 [==============================] - 6s 53ms/step - loss: nan - accuracy: 0.4852 - val_loss: nan - val_accuracy: 0.4891
Epoch 2/20
92/92 [==============================] - 5s 51ms/step - loss: nan - accuracy: 0.4874 - val_loss: nan - val_accuracy: 0.4891
Epoch 3/20
92/92 [==============================] - 5s 55ms/step - loss: nan - accuracy: 0.4874 - val_loss: nan - val_accuracy: 0.4891
Epoch 4/20
16/92 [====>.........................] - ETA: 3s - loss: nan - accuracy: 0.4688

KeyboardInterrupt: 